In [81]:

################################
# Analise de ordens planejadas #
################################


#######################
# Algo Libraries
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import altair as alt
import matplotlib.pyplot as plt

# Excel libraries
from openpyxl import Workbook
from openpyxl.drawing.image import Image

# Image libraries
from IPython.display import Image as IPythonImage

#######################
# Constantes de classificaçao pre-analise
CV_THRESHOLD = 0.7
TMD_THRESHOLD = 1.32

# Valor em que se considera alto para alterar a politica para ZE
ORDER_ZE_TRESHOLD = 500

# para atender incertezas de tempo de reposição e demanda
GORDURA_PR = 0  # % a mais que o 2 consumo nos ult 12 LT
GORDURA_MAX= 0  # % a mais que a media de consumo anual dos ultimos 3 anos

# Valor de OP considerado baixo
BAIXO_VALOR = 100

# Volume alto em CM³ de OP
ALTO_VOLUME = 10000000 

# Tempo no qual o valor registrado pode ser considerado desatualizado
ANOS_SEM_OC = 3

# janela de analise de consumo em anos
DEMAND_WINDOW = 3

# Tamanho da janela de analise dos leads times
LT_WINDOW = 12

# Lote economico de compra
k = 6

DEBUG = True
#######################
# data da extração de dados SAP

data_OP = dt.datetime.now().strftime('%Y-%m')
#data_OP = '2024-06'
DIR_PATH = f'./data/{data_OP}/'

# create a new directory
import os
if not os.path.exists(DIR_PATH):
    os.makedirs(DIR_PATH)

if not os.path.exists(f'{DIR_PATH}/fotos'):
    os.makedirs(f'{DIR_PATH}/fotos')  

if not os.path.exists(f'{DIR_PATH}/graficos'):
    os.makedirs(f'{DIR_PATH}/graficos')

print(f'Extrair dados de {data_OP}...')


Extrair dados de 2024-07...


In [82]:

### Definir quais colunas serão utilizadas de cada dataframe ###

## Unico material por linha

# OP        - "Data abertura plan.","Material","Txt.brv.material","Grupo de mercadorias","Setor de atividade","Nº peça fabricante","Planejador MRP","Grupo MRP","Tipo de MRP","Prz.entrg.prev.","Estoque total","Ponto reabastec.","Estoque máximo","Estoque de segurança","Valor total","CMM","Demanda","Demanda Med.","Preço Unit.","Qtd. RTP1","Qtd. RTP2","Qtd. RTP3","Qtd. RTP6","Sld. Virtual","Qtd.ordem planejada ","Valor Total","Responsável","Criticidade","Qtd. LMR","Dem. Pro.","Dt. Ult. Pedido","Fornecedor","Nome","Dt. Ult. Requisição","Qtd. Pedido","Qtd. Requisição","Qtd. RemCG","Dt. Ult. 201","Qt. 201 - 12 Meses","1LTD","2LTD","3LTD","4LTD","5LTD"
# 130       - Material, Prz.entrg.prev., 1 LTD,2 LTD,3 LTD,4 LTD,5 LTD,6 LTD,7 LTD,8 LTD,9 LTD,10 LTD,11 LTD,12 LTD,13 LTD,14 LTD,15 LTD,16 LTD,17 LTD,18 LTD,19 LTD,20 LTD,21 LTD,22 LTD,23 LTD,24 LTD,25 LTDutf-8
# 0053      - Material, Peso bruto, Volume

## Mais de um material por linha (join)
# 0028      - Material, Tipo de reserva, Centro custo,data base,Nome do usuário,Cód. Localizpre-analise,Descrição do Equipamento,Material,Texto,Com registro final,Item foi eliminado,Motivo da Reserva
# 0127    - Material, Texto OBS - pt,Texto DB - pt

## Verificar se os arquivos existem

for file in ['OP.XLSX', '0130.XLSX', '0053.XLSX', '0028.XLSX', '0127.XLSX']:
    if not os.path.exists(f'{DIR_PATH}{file}'):
        print(f'Arquivo {file} não encontrado')

## Carregar dados com as colunas selecionadas de arquivos em excel
def load_data():
    
    op      = pd.read_excel(f'{DIR_PATH}OP.XLSX',           usecols=["Data abertura plan.","Material","Txt.brv.material","Grupo de mercadorias","Setor de atividade","Nº peça fabricante","Planejador MRP","Grupo MRP","Tipo de MRP","Prz.entrg.prev.","Estoque total","Ponto reabastec.","Estoque máximo","Estoque de segurança","Valor total","CMM","Demanda","Demanda Med.","Preço Unit.","Qtd. RTP1","Qtd. RTP2","Qtd. RTP3","Qtd. RTP6","Sld. Virtual","Qtd.ordem planejada","Valor Total","Responsável","Criticidade","Qtd. LMR","Dem. Pro.","Dt. Ult. Pedido","Fornecedor","Nome","Dt. Ult. Requisição","Qtd. Pedido","Qtd. Requisição","Qtd. RemCG","Dt. Ult. 201","Qt. 201 - 12 Meses"])
    t0053   = pd.read_excel(f'{DIR_PATH}0053.XLSX',         thousands='.', decimal=',', usecols=['Material','Peso bruto','Volume'])
    t0028   = pd.read_excel(f'{DIR_PATH}0028.XLSX',         usecols=['Material','Tipo de reserva','Centro custo','Data base','Nome do usuário','Cód. Localização','Descrição do Equipamento','Material','Texto','Com registro final','Item foi eliminado','Motivo da Reserva'])
    t0127  = pd.read_excel(f'{DIR_PATH}0127.XLSX',          usecols=['Material','Texto OBS - pt','Texto DB - pt', 'Texto - pt','Texto REF LMR'])
    t0130    = pd.read_excel(f'{DIR_PATH}0130.XLSX',        nrows=10000,   thousands='.', decimal=',', usecols=['Material','1 LTD','2 LTD','3 LTD','4 LTD','5 LTD','6 LTD','7 LTD','8 LTD','9 LTD','10 LTD','11 LTD','12 LTD','13 LTD','14 LTD','15 LTD'])
    
    return op, t0130, t0053, t0028, t0127

op, t0130, t0053, t0028, t0127 = load_data()


In [87]:
### Ajustar e limpar dataframes ###

## Converter Material para string
for df in [op, t0130, t0053, t0028, t0127]:
    df['Material'] = df['Material'].astype(str)

t0053['Material'] = t0053['Material'].str.replace('.0','')

df = op
## Juntar dataframes com um material por linha
df = df.merge(t0130, on='Material', how='left')
df = df.merge(t0053, on='Material', how='left')

# Converter para string
strs = ['Grupo de mercadorias','Setor de atividade','Nº peça fabricante','Planejador MRP','Grupo MRP','Tipo de MRP','Responsável','Criticidade','Fornecedor','Nome','Criticidade']
for s in strs:
    df[s] = df[s].astype(str)

# Converter colunas para datetime DD/MM/YYYY
datas = ['Data abertura plan.','Dt. Ult. Pedido','Dt. Ult. 201','Dt. Ult. Requisição']
for data in datas:
    df[data] = df[data].apply(pd.to_datetime, format='%d.%m.%Y')

# round up lts
lts = ['1 LTD','2 LTD','3 LTD','4 LTD','5 LTD','6 LTD','7 LTD','8 LTD','9 LTD','10 LTD','11 LTD','12 LTD','13 LTD','14 LTD','15 LTD']
for lt in lts:
    df[lt] = np.ceil(df[lt])

# Volume e peso por OP convertidos para int e fillna(0)
df['Volume da OP']      = (df['Volume'] * df['Qtd.ordem planejada']).fillna(0).astype(int)
df['Peso bruto da OP']  = (df['Peso bruto'] * df['Qtd.ordem planejada']).fillna(0).astype(int)

df['pre-analise'] = ''
df['Image'] = ''

#df.info()


In [84]:
### Funçoes auxiliares ###

# Calcular tempo medio entre consumos = quantidade de consumos nao zero dividido pela quantidade de consumo total
# as colunas com LTD representam consumos
# considerar os primeiros LT_WINDOW (tamanho da janela de analise)
def TMD(df):
    consumo_LT = df[lts]
    consumo_LT = consumo_LT[:LT_WINDOW]
    
    non_zero   = consumo_LT[consumo_LT!=0].count()
    total      = consumo_LT.count()
    # TMD = total time between consumption / non zero consumption
    # se non_zero = 0, TMD = -1 ou seja, não tem consumo diferente de zero
    tmd        = total/non_zero if non_zero != 0 else -1
    return tmd

# Calcular coeficiente de variação
def CV(df):
    consumo_LT = df[lts]
    consumo_LT = consumo_LT[:LT_WINDOW]
    
    std  = np.std(consumo_LT, ddof=0)
    mean = consumo_LT.mean()
    # CV = desvio padrão / média 
    # se media = 0, cv = -1, ou seja, nao tem consumo diferente de 0
    cv = std/mean if mean !=0 else -1
    return cv

# Classificar por consumo
def classify(df):

    if df['TMD'] ==-1 or df['CV'] == -1:
        return 'Apenas consumo zero'
    # CV² consumption > CV_THRESHOLD and Average time between consumption > TMD_THRESHOLD = Esporádico
    elif df['CV'] > CV_THRESHOLD and df['TMD'] > TMD_THRESHOLD:
        return 'Esporádico'
    # CV² consumption > CV_THRESHOLD and Average time between consumption < TMD_THRESHOLD = Intermitente  
    elif df['CV'] > CV_THRESHOLD and df['TMD'] < TMD_THRESHOLD:
        return 'Intermitente'
    # CV² consumption < CV_THRESHOLD and Average time between consumption > TMD_THRESHOLD = Erratico 
    elif df['CV'] < CV_THRESHOLD and df['TMD'] > TMD_THRESHOLD:
        return 'Errático'
    # CV² consumption < CV_THRESHOLD and Average time between consumption < TMD_THRESHOLD = Suave
    elif df['CV'] < CV_THRESHOLD and df['TMD'] < TMD_THRESHOLD:
        return 'Suave'
    else:
        return 'Outro'

# Buscar reservas (ultimas n):
def buscar_reservas(material,n):
    reservas = t0028[t0028['Material'] == material]
    reservas = reservas[reservas['Com registro final'] == 'X']
    reservas = reservas[reservas['Tipo de reserva'] == 1]
    reservas = reservas[["Centro custo","Data base","Nome do usuário","Cód. Localização","Descrição do Equipamento","Motivo da Reserva"]]    
    reservas = reservas.fillna('-')
    return reservas.head(n)

# Buscar OCs (ultimas n):
def buscar_oc(material,n):
    ocs = oc[oc['Material'] == material]
    # ordenar por data do documento
    ocs = ocs.sort_values(by='Data do documento', ascending=False)

    # filtrar ocs que tenham cancelado no texto
    ocs = ocs[~ocs['Texto breve'].str.contains('cancelado', case=False, na=False)]
    return ocs.head(n)

# Buscar texto observpre-analise
def buscar_obs(material):
    #join all lines in the text
    obs = t0127[t0127['Material'] == material]['Texto OBS - pt'].str.cat(sep='\n')
    return obs

# Buscar descritivo
def buscar_descritivo(material):
    #join all lines in the text
    descritivo = t0127[t0127['Material'] == material]['Texto - pt'].str.cat(sep='\n')
    return descritivo

## DEFINIR NIVEL PR ##
# PR = segundo maior consumo nos ultimos 12 Lead times
# excluidos os outliers
def PR(df):
    consumo_LT = df[lts]
    consumo_LT = consumo_LT[:LT_WINDOW]
    
    consumo_LT = consumo_LT.sort_values(ascending=False)
    
    #detectar outliers: algum valor supera o 3 quartil mais 1.5 vezes o IQR
    Q1 = consumo_LT.quantile(0.25)
    Q3 = consumo_LT.quantile(0.75)
    IQR = Q3 - Q1
    outliers = consumo_LT[consumo_LT > Q3 + 1.5*IQR]

    # quantidade de zeros
    zeros = consumo_LT[consumo_LT == 0].count()
    # se tiver outliers, consumo do segundo maior maior que 2, menos de 6 zeros e o smaller outlier > pr
    if len(outliers) > 0 and consumo_LT.iloc[1] > 2 and zeros<6 and outliers.min() > df['Ponto reabastec.']:
        #print(f'\n**Outliers detectados para {df["Material"]}**')
        #print(outliers)
        # remove outliers
        consumo_LT = consumo_LT[consumo_LT < Q3 + 1.5*IQR]
    
    outliers = outliers.tolist()
    # PR = segundo maior consumo nos ultimos 12 Lead times
    if consumo_LT.iloc[1] == 0:
        return 1,outliers
    else:
        return np.ceil(consumo_LT.iloc[1]*(1+GORDURA_PR/100)),outliers

## DEFINIR MAXIMO ##
# Maximo = consumo anual medio dos ultimos 3 anos 
# PrzEntPrev representa o lead time
# LEC = raiz(2*Demanda anual*Cpedido/Carmazenagem)
# Janela de analise dos ultimos N anos
def maximo(df):
    consumo_LT = df[lts]
    consumo_LT = consumo_LT[:LT_WINDOW]
    lt = df['Prz.entrg.prev.']
    # max = consumo anual medio dos ultimos 3 anos
    if lt%30 == 0:
        max_value = consumo_LT.iloc[0:int(DEMAND_WINDOW*12/(lt/30))].sum()/DEMAND_WINDOW
    else:
        df['pre-analise'] += f'| **Lead time de {lt} fora do esperado para {df["Material"]}**\n'
        max_value = 0
    return np.ceil(max_value*(1+GORDURA_MAX/100))

# Plotar grafico de consumo
def plot_consumo(df):

    if not os.path.exists(f"{DIR_PATH}\\graficos\\{df['Material']}"):

        consumo_LT = df[lts].fillna(0)
        
        # Configuração do gráfico
        fig, ax = plt.subplots()
        
        if consumo_LT.max()!=0:
            # Definir limite máximo do eixo y
            ax.set_ylim(0, max(consumo_LT.max()*1.1,df['Estoque máximo']*1.1))
        else:
            ax.set_ylim(0, 5)   

        # Plotar o consumo
        ax.plot(consumo_LT.T, marker='o', linestyle='-', color='b', label='Consumo')

        # Linhas horizontais para PR e Estoque Máximo
        ax.axhline(df['Ponto reabastec.'], color='red', linestyle='--', label=f'PR  = {df["Ponto reabastec."]}')
        ax.axhline(df['Estoque máximo'], color='black', linestyle='--', label=f'MAX = {df["Estoque máximo"]}')

        # Adicionar rótulos e título
        ax.set_xlabel(f'Lead time: {df["PrzEntPrev"]}')
        ax.set_ylabel('Consumo')
        ax.set_title(f'{df["Material"]} - {df["Txt.brv.material"]}')

        # Inclinar os xticks
        plt.xticks(rotation=45)

        # Adicionar legenda
        ax.legend()

        # Salvar o gráfico
        fig.savefig(f'{DIR_PATH}\\graficos\\{df["Material"]}.png')

        # Limpar a figura para evitar problemas de plotagem futura
        fig.clf()
    else:
        print('Grafico ja gerado')




In [88]:

# Definição de politica e niveis de estoque
def definir_recomendacao(df):
    valor_atualizado    = False
    outliers            = []
    df['pre-analise']   = ''
    df['PR sug']        = ''
    df['MAX sug']       = ''
    df['impacto $']     = 0
    df['descritivo']    = buscar_descritivo(df['Material'])
    df['obs']           = buscar_obs(df['Material'])
    df['tendencia']     = ''
    df['Image']         = f"P:\\Mfotos\\Padronizadas\\{df['Material']}(A).jpg"

    pr_value = 0
    max_value = 0

    if df['Grupo MRP'] == 'ZSTK':
        if df['Classificação']!= 'Apenas consumo zero':
            if df['Dt. Ult. Pedido'] == -1:
                df['pre-analise'] += f'| Nunca comprou\n'
            #comparar data da ultima OC com a data de hoje
            elif (dt.datetime.now() - pd.to_datetime(df['Dt. Ult. Pedido'])).days > ANOS_SEM_OC*360:
                #print('Sem OCs nos ultimos anos, verificar preço unitario')
                df['pre-analise'] += f'| Sem OCs nos ultimos {ANOS_SEM_OC} anos, verificar preço unitario\n'
            else:
                valor_atualizado = True            
            # Se Tipo de MRP = ZM e Classificpre-analise = Suave e Demanda = F entao Tipo de MRP = ZP
            if df['Tipo de MRP'] != 'ZP' and df['Classificação'] == 'Suave':
                df['pre-analise'] += f'| Alterar {df["Material"]} para ZP\n'
                #df['Tipo de MRP'] = 'ZP'
                #print('\nOBS: Para alterar para ZP voce precisa\n- Alterar no MRP4 a previsao de N para D\n- Ajustar na MM17 o CMM')
            # se ZP e demanda != F, alterar para ZM
            if df['Tipo de MRP'] == 'ZP'and df['Volume da OP'] > ALTO_VOLUME:
                    df['pre-analise'] += f'| Volume da OP alto, verificar se há necessidade de parcelamento\n'
            if df['Tipo de MRP'] == 'ZP' and df['Classificação'] != 'Suave':
                df['pre-analise'] += f'| Alterar {df["Material"]} para ZM\n'
                #df['Tipo de MRP'] = 'ZM'
            # Se Ponto reabastec. < 5 e Valor Total > 500 e Criticidade > 0 e Tipo de MRP != ZE e Classificpre-analise != Apenas consumo zero entao Tipo de MRP = ZE
            if df['Classificação']=='Errático' and df['Valor Total'] > ORDER_ZE_TRESHOLD and df['Criticidade'] > 0 and df['Tipo de MRP']  != 'ZE' :
                df['pre-analise'] += f'| Alterar {df["Material"]} para ZE\n'
                #df['Tipo de MRP'] = 'ZE'
            # Se Tipo de MRP = ZM ou Tipo de MRP = ZE e Classificpre-analise != Apenas consumo zero entao alterar PR para segundo maior consumo nos ultimos 12 Lead times e MAX para maior consumo anual dos ultimos 3 anos
            if df['Valor Total'] < BAIXO_VALOR:
                if (df['Tipo de MRP'] == 'ZM'):

                    # O PR sempre é calculado igual
                    pr_value,outliers = PR(df)

                    # se o valor da ordem for muito baixo e o valor da ordem estiver atualizado, aumentar o max para 3 anos de consumo
                    if valor_atualizado:
                        max_value = pr_value + 3*maximo(df)
                    else:
                        df['pre-analise'] += f'| Valor baixo desatualizado\n'
                        
                        # o maximo é o maior entre pr_value+retorno da função e o dobro de pr_value
                        max_value = max(pr_value + maximo(df), pr_value*2)

                elif  df['Tipo de MRP'] == 'ZE':
                    
                    max_value = pr_value
                    if pr_value != df['Ponto reabastec.']:
                        df['pre-analise'] += f'| Sugestao PR={pr_value}\n'
            
            else:    
                max_value = maximo(df)+pr_value

            # se o pr_value ou max for diferente do atual, sugerir a mudança            
            if pr_value != df['Ponto reabastec.']:
                df['pre-analise'] += f'| Sugestao PR={pr_value}\n'
            if max_value != df['Estoque máximo']:
                df['pre-analise'] += f'| Sugestao MAX={max_value}\n'
            
            df['PR sug'] = pr_value
            df['MAX sug'] = max_value
            # estoque medio = (max-pr)/2 + estoque de segurança
            # o impacto é a diferença de estoque medio vezes o preço unitario
            try:
                df['impacto $'] = int(((max_value-pr_value)-(df['Estoque máximo']-df['Ponto reabastec.']))/2*(df['Preço Unit.']))
            except:
                df['impacto $'] = 0
            #Reserva mais saldo > PR
            if df['Qtd. RTP6']+df['Sld. Virtual'] > df['Ponto reabastec.']:
                df['pre-analise'] += f'| Verificar reserva\n'

            if df['Nº peça fabricante'] == '':
                print(buscar_obs(df['Material'])) 
                df['pre-analise'] += f'| Necessario preencher referencia\n'
            
            #media dos LT 1 a 5
            med1 = df[lts[0:4]].mean()
            if med1 > 2:
                if df['1 LTD']<med1 and df['2 LTD']<med1:
                    df['tendencia'] = 'tendencia de Baixa'
                elif df['1 LTD']>med1 and df['2 LTD']>med1:
                    df['tendencia'] = 'tendencia de Alta'
                else:
                    df['tendencia'] = 'Estavel'
            else:
                df['tendencia'] = '-'
            
            if len(outliers) > 0:
                df['pre-analise'] += f'| Outliers detectados {outliers}\n'
                #print(outliers)
        else:
            df['pre-analise'] += f'| Apenas consumo zero\n'

    elif df['Grupo MRP'] == 'SMIT':
        df['pre-analise'] += "| Verificar consulta"
    elif df['Grupo MRP'] == 'FRAC':
        if df['Qtd. LMR']:
            df['pre-analise'] = f"| Abrir consulta para SMIT\nPrezados, \nO codigo {df['Txt.brv.material']} - {df['Material']} teve a licitação fracassada\nFavor indicar uma referencia substituta.\n\nReferencia atual: {df['Nº peça fabricante']}\nUltimas Reservas\n{buscar_reservas(df['Material'],3).to_string(index=False)}"
        else:
            df['pre-analise'] += f'| Sem LMR, Revisar descritivo ou encontrar referencia atualizada'  
    elif df['Grupo MRP'] == 'AD':
        df['pre-analise'] += f"| Solicitar cotação do material com os fornecedores do grupo {df['Grupo de mercadorias']}\n\n"+f"Prezado [Fornecedor],\nSolicito cotação de {df['Qtd.ordem planejada']} unidades do material:\n\n{buscar_descritivo(df['Material'])}"
    elif df['Grupo MRP'] == 'ZSTK' and df['Tipo de MRP'] == 'ZS':    
        df['pre-analise'] += f"| Prezados, \n Solicito verificar a necessidade de compra do material {df['Txt.brv.material']} - {df['Material']}\n\nUltimas Reservas\n{buscar_reservas(df['Material'],3).to_string(index=False)}"  

    return df

    
### Aplicar funções auxiliares ###
df.info()
# Calculate TMD e CV
df['TMD']   = df.apply(TMD, axis=1)
df['CV']    = df.apply(CV , axis=1)

# Classificpre-analise
df['Classificação'] = df.apply(classify, axis=1)

#df['Abertura'] = pd.to_datetime(df['Abertura'], errors='coerce', format='%d/%m/%Y')

## Dias em op
df['Dias em OP'] = (dt.datetime.now() - df['Data abertura plan.']).dt.days

### Definir pre-analise ###
df = df.apply(definir_recomendacao, axis=1)

### Plotar graficos de consumo ###
#df.apply(plot_consumo, axis=1)
# OP        - "Data abertura plan.","Material","Txt.brv.material","Grupo de mercadorias","Setor de atividade","Nº peça fabricante","Planejador MRP","Grupo MRP","Tipo de MRP","Prz.entrg.prev.","Estoque total","Ponto reabastec.","Estoque máximo","Estoque de segurança","Valor total","CMM","Demanda","Demanda Med.","Preço Unit.","Qtd. RTP1","Qtd. RTP2","Qtd. RTP3","Qtd. RTP6","Sld. Virtual","Qtd.ordem planejada planejada","Valor Total","Responsável","Criticidade","Qtd. LMR","Dem. Pro.","Dt. Ult. Pedido","Fornecedor","Nome","Dt. Ult. Requisição","Qtd. Pedido","Qtd. Requisição","Qtd. RemCG","Dt. Ult. 201","Qt. 201 - 12 Meses","1LTD","2LTD","3LTD","4LTD","5LTD"

# reorder columns
colunas = ['pre-analise','Dias em OP',"Image",'Material','Txt.brv.material','Planejador MRP','Grupo de mercadorias','Grupo MRP','Criticidade','Classificação','Nº peça fabricante','Valor Total','Sld. Virtual','Prz.entrg.prev.','Tipo de MRP','Ponto reabastec.','Estoque máximo','PR sug','MAX sug','impacto $','Qtd.ordem planejada','Preço Unit.','Dt. Ult. Pedido','tendencia','1 LTD','2 LTD','3 LTD','4 LTD','5 LTD','6 LTD','7 LTD','8 LTD','9 LTD','10 LTD','11 LTD','12 LTD','13 LTD','14 LTD','15 LTD','Qtd. RTP1','Qtd. RTP3','Qtd. RTP6','Qtd. LMR','Volume da OP','Setor de atividade','TMD','CV','obs','descritivo']
df_atual = df[colunas]

df_atual.to_excel(f'{DIR_PATH}OP_analisado.xlsx', index=False,freeze_panes=(1,0))

# wait for the file to be created
import time
time.sleep(3)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 60 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Data abertura plan.   486 non-null    datetime64[ns]
 1   Material              486 non-null    object        
 2   Txt.brv.material      486 non-null    object        
 3   Grupo de mercadorias  486 non-null    object        
 4   Setor de atividade    486 non-null    object        
 5   Nº peça fabricante    486 non-null    object        
 6   Planejador MRP        486 non-null    object        
 7   Grupo MRP             486 non-null    object        
 8   Tipo de MRP           486 non-null    object        
 9   Prz.entrg.prev.       486 non-null    int64         
 10  Estoque total         486 non-null    float64       
 11  Ponto reabastec.      486 non-null    float64       
 12  Estoque máximo        486 non-null    float64       
 13  Estoque de segurança

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
#plotar graficos

In [ ]:
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils import get_column_letter
from PIL import Image as PILImage
from PIL import ImageOps
from openpyxl.styles import Alignment

data_OP = dt.datetime.now().strftime('%Y-%m')
width = 100
height = 100
temp_image_path = f"C:\\Users\\pedrohvb\\Documents\\Drafts\\OP_analysis\\data\\{data_OP}\\fotos\\"


# Função para redimensionar a imagem e salvar temporariamente
def resize_image(image_path, width, height, output_path):
    with PILImage.open(image_path) as img:
        resized_img = img.resize((width, height), PILImage.Resampling.LANCZOS)
        resized_img.save(output_path)
    return output_path

# Carregar o workbook existente
wb = load_workbook(f'{DIR_PATH}OP_analisado.xlsx')

try:
    # Selecionar a planilha ativa ou especificar uma planilha
    ws = wb.active 

    # leia as celulas da coluna C e adicione a imagem
    for i, row in enumerate(ws['C'], start=1):
        if i==1:
            continue
    
        print(row.value)
        try:
            resized_image_path = resize_image(row.value, width, height, temp_image_path+row.value.split('\\')[-1])
            #resized_image_path = temp_image_path+row.value.split('\\')[-1]
            print(resized_image_path)
            row.value = ''
            # Adicionar a imagem ao Excel
            img = Image(resized_image_path)
            ws.add_image(img, f'C{i}',)
        except Exception as e:
            print(e)
            print(f"{row.value} - imagem não encontrada")
        
    # Ajustar a largura das colunas e a altura das linhas
    for column_cells in ws.columns:
        max_length = 0
        column = get_column_letter(column_cells[0].column)
        for cell in column_cells:
            cell.alignment = Alignment(horizontal='center', vertical='center')
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = max(adjusted_width,15)

    for row_cells in ws.iter_rows():
        max_height = 0
        for cell in row_cells:
            cell.alignment = Alignment(horizontal='center', vertical='center')
            if cell.value:
                max_height = max(max_height, len(str(cell.value)))
        adjusted_height = (max_height + 2) * 1.2

        print('row =',row)
        if row > 1:
            ws.row_dimensions[cell.row].height = height
            #ws.row_dimensions[cell.row].height = max(adjusted_height,100)

        
    # Salvar o workbook
    wb.save(f'{DIR_PATH}OP_analisado_com_foto.xlsx')
except Exception as e:
    print(e)
    print('Erro ao adicionar imagem')
    wb.save(f'{DIR_PATH}OP_analisado_com_foto.xlsx')

P:\Mfotos\Padronizadas\116076(A).jpg
C:\Users\pedrohvb\Documents\Drafts\OP_analysis\data\2024-07\fotos\116076(A).jpg
P:\Mfotos\Padronizadas\287830(A).jpg
C:\Users\pedrohvb\Documents\Drafts\OP_analysis\data\2024-07\fotos\287830(A).jpg
P:\Mfotos\Padronizadas\287962(A).jpg
C:\Users\pedrohvb\Documents\Drafts\OP_analysis\data\2024-07\fotos\287962(A).jpg
P:\Mfotos\Padronizadas\163759(A).jpg
C:\Users\pedrohvb\Documents\Drafts\OP_analysis\data\2024-07\fotos\163759(A).jpg
P:\Mfotos\Padronizadas\287830(A).jpg
C:\Users\pedrohvb\Documents\Drafts\OP_analysis\data\2024-07\fotos\287830(A).jpg
P:\Mfotos\Padronizadas\529125(A).jpg
C:\Users\pedrohvb\Documents\Drafts\OP_analysis\data\2024-07\fotos\529125(A).jpg
P:\Mfotos\Padronizadas\236900(A).jpg
C:\Users\pedrohvb\Documents\Drafts\OP_analysis\data\2024-07\fotos\236900(A).jpg
P:\Mfotos\Padronizadas\332313(A).jpg
C:\Users\pedrohvb\Documents\Drafts\OP_analysis\data\2024-07\fotos\332313(A).jpg
P:\Mfotos\Padronizadas\327905(A).jpg
C:\Users\pedrohvb\Documents